<a href="https://colab.research.google.com/github/srikanthmandava9/Srikant_INFO5731_Spring2021/blob/master/In_class_exercise_08_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The eighth in-class-exercise (20 points in total, 3/30/2021)**

The data for this exercise is from the dataset you created from assignment three. Please perform answer the following questions based on your data:

## (1) (10 points) Write a python program to extract the sentiment related terms from the corpus. You may use python package such as polyglot or external lexicon resources in the question. Rank the sentiment related terms by frequency.

In [55]:
!pip install polyglot

In [52]:
!pip install pyicu

In [53]:
!pip install morfessor

In [56]:
!pip install pycld2

In [57]:
%%bash
polyglot download sentiment2.en

[polyglot_data] Downloading package sentiment2.en to
[polyglot_data]     /root/polyglot_data...
[polyglot_data]   Package sentiment2.en is already up-to-date!


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import pandas as pd
df = pd.read_csv('https://github.com/srikanthmandava9/Srikant_INFO5731_Spring2021/raw/master/training_data.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [ ]:
from nltk.corpus import stopwords
from textblob import TextBlob
df['clean_text'] = df['clean_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['clean_text'] = df['clean_text'].str.replace('[^\w\s]','')
stop = stopwords.words('english')
df['clean_text'] = df['clean_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['clean_text'] = df['clean_text'].apply(lambda x: str(TextBlob(x).correct()))

In [ ]:
from polyglot.text import Text
sentimental_words = []
for line in df['clean_text']:
  text = Text(line)
  for w in text.words:
    if w.polarity != 0:
      sentimental_words.append(w)
sentimental_words

['perfect',
 'crazy',
 'haunt',
 'worth',
 'like',
 'mock',
 'helpless',
 'pure',
 'entertain',
 'decent',
 'twist',
 'well',
 'cool',
 'terrific',
 'love',
 'good',
 'well',
 'appeal',
 'miss',
 'better',
 'like',
 'love',
 'like',
 'faith',
 'worth',
 'enjoy',
 'master',
 'brilliant',
 'perfect',
 'love',
 'promise',
 'master',
 'break',
 'promise',
 'master',
 'clash',
 'antagonist',
 'clash',
 'love',
 'antagonist',
 'brilliant',
 'fail',
 'antagonist',
 'devil',
 'prowess',
 'expel',
 'fancy',
 'rejoice',
 'broke',
 'broke',
 'master',
 'desert',
 'extraordinary',
 'work',
 'master',
 'best',
 'master',
 'master',
 'master',
 'love',
 'hate',
 'best',
 'worth',
 'love',
 'master',
 'extraordinary',
 'charm',
 'steal',
 'great',
 'vile',
 'thrill',
 'master',
 'work',
 'perfect',
 'critic',
 'perfect',
 'love',
 'good',
 'slow',
 'safe',
 'ruthless',
 'well',
 'master',
 'best',
 'kill',
 'steal',
 'plea',
 'miss',
 'recommend',
 'well',
 'appeal',
 'miss',
 'better',
 'like',
 'lo

In [ ]:
from collections import Counter
frequencies = Counter(sentimental_words)
rank_df = pd.DataFrame(list(frequencies.most_common()), columns=['Words', 'Frequencies'])
rank_df.index = list(range(1, len(frequencies.most_common())+1))
rank_df


,Words,Frequencies
1,master,13
2,love,12
3,like,8
4,well,7
5,good,7
...,...,...
68,angel,1
69,ignore,1
70,flaw,1
71,deadly,1


## (2) (10 points) Compare the performance of the following tools in sentiment identification: TextBlob (https://textblob.readthedocs.io/en/dev/), VADER (https://github.com/cjhutto/vaderSentiment), TFIDF-based Support Vector Machine (SVM) (Split your data into training and testing data). Take your own annotation as the standard answers. 

Reference code: https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4

In [ ]:
def Encoder(senti):
  sentiment_encoded = 0
  if senti == 'Positive':
    sentiment_encoded = 1
  elif senti == 'Negative':
    sentiment_encoded = 2
  return sentiment_encoded

In [ ]:
actual_sentiment = []
for senti in df['sentiment']:
  actual_sentiment.append(Encoder(senti))

In [ ]:
from textblob import TextBlob
textblob_sentiments = []
for line in df['clean_text']:
  polarity = TextBlob(line).sentiment.polarity
  if polarity > 0:
    textblob_sentiments.append(1)
  elif polarity < 0:
    textblob_sentiments.append(2)
  elif polarity == 0.0:
    textblob_sentiments.append(0)
text_blob_df = pd.DataFrame(list(zip(df['clean_text'], actual_sentiment, textblob_sentiments)), columns = ['Reviews', 'Actual', 'Predicted'])
text_blob_df


,Reviews,Actual,Predicted
0,perfect characterise vital v half little bit l...,1,1
1,twist turn straight move lot mass content acto...,2,1
2,story cometh differ somewhat hope well made co...,0,1
3,awesom move vera level big treat thalapathi fa...,1,1
4,master brilliant example perfect cat vous acti...,1,1
5,wast money time ff shout worth allow hope rate...,2,1
6,review post review float world master became1s...,1,1
7,st half thalapathi attitude fight vaathi come ...,0,1
8,critic rate low differ vital act move perfect ...,0,1
9,first big appreci go vital sethupathi act vill...,1,1


In [ ]:
from sklearn.metrics import f1_score, accuracy_score
text_blob_accuracy = accuracy_score(text_blob_df['Actual'], text_blob_df['Predicted'])*100
text_blob_f1_score = f1_score(text_blob_df['Actual'], text_blob_df['Predicted'], average = 'macro')
print("Text Blob Accuracy is {0} and F1 Score is {1}".format(text_blob_accuracy, text_blob_f1_score))

Text Blob Accuracy is 50.0 and F1 Score is 0.2222222222222222


In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()
vader_sentiments = []
for line in df['clean_text']:
  polarity = vader.polarity_scores(line)['compound']
  if polarity > 0:
    vader_sentiments.append(1)
  elif polarity < 0:
    vader_sentiments.append(2)
  elif polarity == 0.0:
    vader_sentiments.append(0)
vader_df = pd.DataFrame(list(zip(df['clean_text'], actual_sentiment, vader_sentiments)), columns = ['Reviews', 'Actual', 'Predicted'])
vader_df

,Reviews,Actual,Predicted
0,perfect characterise vital v half little bit l...,1,2
1,twist turn straight move lot mass content acto...,2,1
2,story cometh differ somewhat hope well made co...,0,1
3,awesom move vera level big treat thalapathi fa...,1,1
4,master brilliant example perfect cat vous acti...,1,1
5,wast money time ff shout worth allow hope rate...,2,1
6,review post review float world master became1s...,1,1
7,st half thalapathi attitude fight vaathi come ...,0,2
8,critic rate low differ vital act move perfect ...,0,1
9,first big appreci go vital sethupathi act vill...,1,1


In [ ]:
from sklearn.metrics import f1_score, accuracy_score
vader_accuracy = accuracy_score(vader_df['Actual'], vader_df['Predicted'])*100
vader_f1_score = f1_score(vader_df['Actual'], vader_df['Predicted'], average = 'macro')
print("Vader Accuracy is {0} and F1 Score is {1}".format(vader_accuracy, vader_f1_score))

Vader Accuracy is 50.0 and F1 Score is 0.2380952380952381


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
TFid_vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

In [ ]:

Train_X, Test_X, Train_Y, Test_Y = train_test_split(df['clean_text'],df['sentiment'],test_size=0.33)
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)
Train_X_vectors = TFid_vectorizer.fit_transform(Train_X)
Test_X_vectors = TFid_vectorizer.transform(Test_X)

In [ ]:
from sklearn import svm
from sklearn.metrics import classification_report
svm_model = svm.SVC(kernel='linear')
svm_model.fit(Train_X_vectors, Train_Y)
predicted = svm_model.predict(Test_X_vectors)
report = classification_report(Test_Y, predicted, output_dict=True)
report


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'0': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 2},
 '1': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 2},
 '2': {'f1-score': 0.6,
  'precision': 0.42857142857142855,
  'recall': 1.0,
  'support': 3},
 'accuracy': 0.42857142857142855,
 'macro avg': {'f1-score': 0.19999999999999998,
  'precision': 0.14285714285714285,
  'recall': 0.3333333333333333,
  'support': 7},
 'weighted avg': {'f1-score': 0.2571428571428571,
  'precision': 0.18367346938775508,
  'recall': 0.42857142857142855,
  'support': 7}}